stratifiedとGroupcvをアンサンブルアンサンブル
histgram系のを抜く
image featuresを追加 sharpnessなど
lgbmとxgbでweighting
Target Encodingする
seed ensenbleする
Oputuna した パラメータを試す
#  Forked from [Baseline Modeling](https://www.kaggle.com/wrosinski/baselinemodeling)

## Added Image features from [Extract Image features from pretrained NN](https://www.kaggle.com/christofhenkel/extract-image-features-from-pretrained-nn)

## Added Image size features from [Extract Image Features](https://www.kaggle.com/kaerunantoka/extract-image-features)

In [ ]:
import gc
import glob
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
import warnings

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from joblib import Parallel, delayed
from tqdm import tqdm, tqdm_notebook

%matplotlib inline

np.random.seed(seed=1337)
warnings.filterwarnings('ignore')

split_char = '/'

In [ ]:
class Config:
    num_seed_stratified_group = 4
    num_seed_stratified = 4
    num_seed_group = 4
    

In [ ]:
os.listdir('../input')

In [ ]:
train = pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv')
test = pd.read_csv('../input/petfinder-adoption-prediction/test/test.csv')
sample_submission = pd.read_csv('../input/petfinder-adoption-prediction/test/sample_submission.csv')

## Image features

In [ ]:
import cv2
from PIL import Image as IMG
import os
from keras.applications.densenet import preprocess_input, DenseNet121

In [ ]:
img_size = 256
batch_size = 32

In [ ]:
def resize_no_pad(im):
    old_size = im.shape[:2]
    ratio = float(img_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    im = cv2.resize(im, (new_size[1], new_size[0]))
    return im

def load_raw_image(path, pet_id):
    image = cv2.imread(f'{path}{pet_id}-1.jpg')
    new_image = resize_no_pad(image)
    return  new_image, image.shape

from collections import defaultdict
import operator
def color_analysis(img):
    # obtain the color palatte of the image 
    palatte = defaultdict(int)
    for pixel in img.getdata():
        palatte[pixel] += 1
    
    # sort the colors present in the image 
    sorted_x = sorted(palatte.items(), key=operator.itemgetter(1), reverse = True)
    light_shade, dark_shade, shade_count = 0, 0, 0
    pixel_limit = 25
    for i, x in enumerate(sorted_x[:pixel_limit]):
        if all(xx <= 20 for xx in x[0][:3]): ## dull : too much darkness 
            dark_shade += x[1]
        if all(xx >= 220 for xx in x[0][:3]): ## bright : too much whiteness 
            light_shade += x[1]
        shade_count += x[1]
        
    light_percent = round((float(light_shade)/shade_count)*100, 2)
    dark_percent = round((float(dark_shade)/shade_count)*100, 2)
    return light_percent, dark_percent

def get_blurrness_score(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    fm = cv2.Laplacian(image, cv2.CV_64F).var()
    return fm

In [ ]:
from scipy.stats import skew

pet_ids = train['PetID'].values
im_feature_dict = pd.DataFrame(index=pet_ids, columns=['light_percent', 'dark_percent', 'sharpness',
                                                      'top1_size', 'top1_width', 'top1_height']).add_prefix('im_')

for pid in tqdm_notebook(pet_ids):
    try:
        im, original_shape = load_raw_image("../input/petfinder-adoption-prediction/train_images/", pid)
        im_feature_dict.loc[pid, 'im_top1_size'] = original_shape[0] * original_shape[1]
        im_feature_dict.loc[pid, 'im_top1_width'] = original_shape[1]
        im_feature_dict.loc[pid, 'im_top1_height'] = original_shape[0]
        
        im_feature_dict.loc[pid, 'im_sharpness'] = get_blurrness_score(im)
        im = IMG.fromarray(np.uint8(im))
        light_percent, dark_percent = color_analysis(im)
        im_feature_dict.loc[pid, 'im_light_percent'] = light_percent
        im_feature_dict.loc[pid, 'im_dark_percent'] = dark_percent
    except AttributeError:
        pass


pet_ids = test['PetID'].values
test_im_feature_dict = pd.DataFrame(index=pet_ids, columns=['light_percent', 'dark_percent', 'sharpness',
                                                            'top1_size', 'top1_width', 'top1_height']).add_prefix('im_')
for pid in tqdm_notebook(pet_ids):
    try:
        im, original_shape = load_raw_image("../input/petfinder-adoption-prediction/test_images/", pid)
        test_im_feature_dict.loc[pid, 'im_top1_size'] = original_shape[0] * original_shape[1]
        test_im_feature_dict.loc[pid, 'im_top1_width'] = original_shape[1]
        test_im_feature_dict.loc[pid, 'im_top1_height'] = original_shape[0]
        test_im_feature_dict.loc[pid, 'im_sharpness'] = get_blurrness_score(im)
        im = IMG.fromarray(np.uint8(im))
        light_percent, dark_percent = color_analysis(im)
        test_im_feature_dict.loc[pid, 'im_light_percent'] = light_percent
        test_im_feature_dict.loc[pid, 'im_dark_percent'] = dark_percent
    except AttributeError:
        pass

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
im_feature_dict.fillna(-1).hist(ax=ax)

In [ ]:
train = train.merge(im_feature_dict, how='left', left_on='PetID', right_index=True)
print(train.shape)
train.head()

In [ ]:
test = test.merge(test_im_feature_dict, how='left', left_on='PetID', right_index=True)
print(test.shape)
test.head()

In [ ]:
def resize_to_square(im):
    old_size = im.shape[:2]
    ratio = float(img_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    im = cv2.resize(im, (new_size[1], new_size[0]))
    delta_w = img_size - new_size[1]
    delta_h = img_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)
    return new_im

def load_image(path, pet_id):
    image = cv2.imread(f'{path}{pet_id}-1.jpg')
    new_image = resize_to_square(image)
    new_image = preprocess_input(new_image)
    return new_image

In [ ]:
img_size = 256
batch_size = 32

In [ ]:
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Input, Lambda, AveragePooling1D
import keras.backend as K
inp = Input((256,256,3))
backbone = DenseNet121(input_tensor = inp, 
                       weights="../input/densenet-keras/DenseNet-BC-121-32-no-top.h5",
                       include_top = False)
x = backbone.output
x = GlobalAveragePooling2D()(x)
x = Lambda(lambda x: K.expand_dims(x,axis = -1))(x)
x = AveragePooling1D(4)(x)
out = Lambda(lambda x: x[:,:,0])(x)

m = Model(inp,out)

In [ ]:
pet_ids = train['PetID'].values
n_batches = len(pet_ids) // batch_size + 1

features = {}
for b in tqdm(range(n_batches)):
    start = b*batch_size
    end = (b+1)*batch_size
    batch_pets = pet_ids[start:end]
    batch_images = np.zeros((len(batch_pets),img_size,img_size,3))
    for i,pet_id in enumerate(batch_pets):
        try:
            batch_images[i] = load_image("../input/petfinder-adoption-prediction/train_images/", pet_id)
        except:
            pass
    batch_preds = m.predict(batch_images)
    for i,pet_id in enumerate(batch_pets):
        features[pet_id] = batch_preds[i]

In [ ]:
train_feats = pd.DataFrame.from_dict(features, orient='index')
train_feats.columns = [f'pic_{i}' for i in range(train_feats.shape[1])]

In [ ]:
pet_ids = test['PetID'].values
n_batches = len(pet_ids) // batch_size + 1

features = {}
for b in tqdm(range(n_batches)):
    start = b*batch_size
    end = (b+1)*batch_size
    batch_pets = pet_ids[start:end]
    batch_images = np.zeros((len(batch_pets),img_size,img_size,3))
    for i,pet_id in enumerate(batch_pets):
        try:
            batch_images[i] = load_image("../input/petfinder-adoption-prediction/test_images/", pet_id)
        except:
            pass
    batch_preds = m.predict(batch_images)
    for i,pet_id in enumerate(batch_pets):
        features[pet_id] = batch_preds[i]

In [ ]:
test_feats = pd.DataFrame.from_dict(features, orient='index')
test_feats.columns = [f'pic_{i}' for i in range(test_feats.shape[1])]

In [ ]:
train_feats = train_feats.reset_index()
train_feats.rename({'index': 'PetID'}, axis='columns', inplace=True)

test_feats = test_feats.reset_index()
test_feats.rename({'index': 'PetID'}, axis='columns', inplace=True)

In [ ]:
all_ids = pd.concat([train, test], axis=0, ignore_index=True, sort=False)[['PetID']]
all_ids.shape

In [ ]:
n_components = 32
svd_ = TruncatedSVD(n_components=n_components, random_state=1337)

features_df = pd.concat([train_feats, test_feats], axis=0)
features = features_df[[f'pic_{i}' for i in range(256)]].values

svd_col = svd_.fit_transform(features)
svd_col = pd.DataFrame(svd_col)
svd_col = svd_col.add_prefix('IMG_SVD_')

img_features = pd.concat([all_ids, svd_col], axis=1)

## About metadata and sentiment

In [ ]:
labels_breed = pd.read_csv('../input/petfinder-adoption-prediction/breed_labels.csv')
labels_state = pd.read_csv('../input/petfinder-adoption-prediction/color_labels.csv')
labels_color = pd.read_csv('../input/petfinder-adoption-prediction/state_labels.csv')

In [ ]:
train_image_files = sorted(glob.glob('../input/petfinder-adoption-prediction/train_images/*.jpg'))
train_metadata_files = sorted(glob.glob('../input/petfinder-adoption-prediction/train_metadata/*.json'))
train_sentiment_files = sorted(glob.glob('../input/petfinder-adoption-prediction/train_sentiment/*.json'))

print(f'num of train images files: {len(train_image_files)}')
print(f'num of train metadata files: {len(train_metadata_files)}')
print(f'num of train sentiment files: {len(train_sentiment_files)}')


test_image_files = sorted(glob.glob('../input/petfinder-adoption-prediction/test_images/*.jpg'))
test_metadata_files = sorted(glob.glob('../input/petfinder-adoption-prediction/test_metadata/*.json'))
test_sentiment_files = sorted(glob.glob('../input/petfinder-adoption-prediction/test_sentiment/*.json'))

print(f'num of test images files: {len(test_image_files)}')
print(f'num of test metadata files: {len(test_metadata_files)}')
print(f'num of test sentiment files: {len(test_sentiment_files)}')

### Train

In [ ]:
# Images:
train_df_ids = train[['PetID']]
print(train_df_ids.shape)

# Metadata:
train_df_ids = train[['PetID']]
train_df_metadata = pd.DataFrame(train_metadata_files)
train_df_metadata.columns = ['metadata_filename']
train_metadata_pets = train_df_metadata['metadata_filename'].apply(lambda x: x.split(split_char)[-1].split('-')[0])
train_df_metadata = train_df_metadata.assign(PetID=train_metadata_pets)
print(len(train_metadata_pets.unique()))

pets_with_metadatas = len(np.intersect1d(train_metadata_pets.unique(), train_df_ids['PetID'].unique()))
print(f'fraction of pets with metadata: {pets_with_metadatas / train_df_ids.shape[0]:.3f}')

# Sentiment:
train_df_ids = train[['PetID']]
train_df_sentiment = pd.DataFrame(train_sentiment_files)
train_df_sentiment.columns = ['sentiment_filename']
train_sentiment_pets = train_df_sentiment['sentiment_filename'].apply(lambda x: x.split(split_char)[-1].split('.')[0])
train_df_sentiment = train_df_sentiment.assign(PetID=train_sentiment_pets)
print(len(train_sentiment_pets.unique()))

pets_with_sentiments = len(np.intersect1d(train_sentiment_pets.unique(), train_df_ids['PetID'].unique()))
print(f'fraction of pets with sentiment: {pets_with_sentiments / train_df_ids.shape[0]:.3f}')

### Test

In [ ]:
# Images:
test_df_ids = test[['PetID']]
print(test_df_ids.shape)

# Metadata:
test_df_metadata = pd.DataFrame(test_metadata_files)
test_df_metadata.columns = ['metadata_filename']
test_metadata_pets = test_df_metadata['metadata_filename'].apply(lambda x: x.split(split_char)[-1].split('-')[0])
test_df_metadata = test_df_metadata.assign(PetID=test_metadata_pets)
print(len(test_metadata_pets.unique()))

pets_with_metadatas = len(np.intersect1d(test_metadata_pets.unique(), test_df_ids['PetID'].unique()))
print(f'fraction of pets with metadata: {pets_with_metadatas / test_df_ids.shape[0]:.3f}')

# Sentiment:
test_df_sentiment = pd.DataFrame(test_sentiment_files)
test_df_sentiment.columns = ['sentiment_filename']
test_sentiment_pets = test_df_sentiment['sentiment_filename'].apply(lambda x: x.split(split_char)[-1].split('.')[0])
test_df_sentiment = test_df_sentiment.assign(PetID=test_sentiment_pets)
print(len(test_sentiment_pets.unique()))

pets_with_sentiments = len(np.intersect1d(test_sentiment_pets.unique(), test_df_ids['PetID'].unique()))
print(f'fraction of pets with sentiment: {pets_with_sentiments / test_df_ids.shape[0]:.3f}')

## Extract features from json

In [ ]:
class PetFinderParser(object):
    
    def __init__(self, debug=False):
        
        self.debug = debug
        self.sentence_sep = ' '
        
        self.extract_sentiment_text = False
    
    def open_json_file(self, filename):
        with open(filename, 'r', encoding='utf-8') as f:
            json_file = json.load(f)
        return json_file
        
    def parse_sentiment_file(self, file):
        """
        Parse sentiment file. Output DF with sentiment features.
        """
        
        file_sentiment = file['documentSentiment']
        file_entities = [x['name'] for x in file['entities']]
        file_entities = self.sentence_sep.join(file_entities)
        
        file_sentences_sentiment = [x['sentiment'] for x in file['sentences']]
        
        file_sentences_sentiment = pd.DataFrame.from_dict(
            file_sentences_sentiment, orient='columns')
        file_sentences_sentiment_df = pd.DataFrame(
            {
                'magnitude_sum': file_sentences_sentiment['magnitude'].sum(axis=0),
                'magnitude_mean': file_sentences_sentiment['magnitude'].mean(axis=0),
                'magnitude_var': file_sentences_sentiment['magnitude'].var(axis=0),
                'magnitude_max': file_sentences_sentiment['magnitude'].max(axis=0),
                'magnitude_min': file_sentences_sentiment['magnitude'].min(axis=0),
                
                
                'score_sum': file_sentences_sentiment['score'].sum(axis=0),
                'score_mean': file_sentences_sentiment['score'].mean(axis=0),
                'score_var': file_sentences_sentiment['score'].var(axis=0),
                'score_max': file_sentences_sentiment['score'].max(axis=0),
                'score_min': file_sentences_sentiment['score'].min(axis=0),
                
            }, index=[0]
        )
        
        df_sentiment = pd.DataFrame.from_dict(file_sentiment, orient='index').T
        df_sentiment = pd.concat([df_sentiment, file_sentences_sentiment_df], axis=1)
            
        df_sentiment['entities'] = file_entities
        df_sentiment['language'] = file['language']
        
        df_sentiment = df_sentiment.add_prefix('sentiment_')
        
        return df_sentiment
    
    def parse_metadata_file(self, file):
        """
        Parse metadata file. Output DF with metadata features.
        """
        
        file_keys = list(file.keys())
        
        if 'labelAnnotations' in file_keys:
            file_annots = file['labelAnnotations']
            file_top_score = np.asarray([x['score'] for x in file_annots]).mean()
            file_top_score_var = np.asarray([x['score'] for x in file_annots]).var()
            file_top_score_max = np.asarray([x['score'] for x in file_annots]).max()
            
            file_top_desc = [x['description'] for x in file_annots]
        else:
            file_top_score = np.nan
            file_top_score_var = np.nan
            file_top_score_max = np.nan
            file_top_desc = ['']
        
        file_colors = file['imagePropertiesAnnotation']['dominantColors']['colors']
        file_crops = file['cropHintsAnnotation']['cropHints']

        file_color_score = np.asarray([x['score'] for x in file_colors]).mean()
        file_color_score_var = np.asarray([x['score'] for x in file_colors]).var()
        file_color_score_max = np.asarray([x['score'] for x in file_colors]).max()
        
        file_color_pixelfrac = np.asarray([x['pixelFraction'] for x in file_colors]).mean()
        file_color_pixelfrac_var = np.asarray([x['pixelFraction'] for x in file_colors]).var()
        file_color_pixelfrac_max = np.asarray([x['pixelFraction'] for x in file_colors]).max()
        

        file_crop_conf = np.asarray([x['confidence'] for x in file_crops]).mean()
        
        if 'importanceFraction' in file_crops[0].keys():
            file_crop_importance = np.asarray([x['importanceFraction'] for x in file_crops]).mean()
        else:
            file_crop_importance = np.nan

        df_metadata = {
            'annots_score': file_top_score,
            'annots_score_var': file_top_score_var,
            'annots_score_max': file_top_score_max,
            
            'color_score': file_color_score,
            'color_score_var': file_color_score_var,
            'color_score_max': file_color_score_max,
            
            'color_pixelfrac': file_color_pixelfrac,
            'color_pixelfrac_var': file_color_pixelfrac_var,
            'color_pixelfrac_max': file_color_pixelfrac_max,
            
            'crop_conf': file_crop_conf,
            'crop_importance': file_crop_importance,
            'annots_top_desc': self.sentence_sep.join(file_top_desc)
        }
        
        df_metadata = pd.DataFrame.from_dict(df_metadata, orient='index').T
        df_metadata = df_metadata.add_prefix('metadata_')
        
        return df_metadata
    

def extract_additional_features(pet_id, mode='train'):
    
    sentiment_filename = f'../input/petfinder-adoption-prediction/{mode}_sentiment/{pet_id}.json'
    try:
        sentiment_file = pet_parser.open_json_file(sentiment_filename)
        df_sentiment = pet_parser.parse_sentiment_file(sentiment_file)
        df_sentiment['PetID'] = pet_id
    except FileNotFoundError:
        df_sentiment = []

    dfs_metadata = []
    metadata_filenames = sorted(glob.glob(f'../input/petfinder-adoption-prediction/{mode}_metadata/{pet_id}*.json'))
    if len(metadata_filenames) > 0:
        for f in metadata_filenames:
            metadata_file = pet_parser.open_json_file(f)
            df_metadata = pet_parser.parse_metadata_file(metadata_file)
            df_metadata['PetID'] = pet_id
            dfs_metadata.append(df_metadata)
        dfs_metadata = pd.concat(dfs_metadata, ignore_index=True, sort=False)
    dfs = [df_sentiment, dfs_metadata]
    
    return dfs


pet_parser = PetFinderParser()

In [ ]:
debug = False
train_pet_ids = train.PetID.unique()
test_pet_ids = test.PetID.unique()

if debug:
    train_pet_ids = train_pet_ids[:1000]
    test_pet_ids = test_pet_ids[:500]


dfs_train = Parallel(n_jobs=-1, verbose=1)(
    delayed(extract_additional_features)(i, mode='train') for i in train_pet_ids)

train_dfs_sentiment = [x[0] for x in dfs_train if isinstance(x[0], pd.DataFrame)]
train_dfs_metadata = [x[1] for x in dfs_train if isinstance(x[1], pd.DataFrame)]

train_dfs_sentiment = pd.concat(train_dfs_sentiment, ignore_index=True, sort=False)
train_dfs_metadata = pd.concat(train_dfs_metadata, ignore_index=True, sort=False)

print(train_dfs_sentiment.shape, train_dfs_metadata.shape)


dfs_test = Parallel(n_jobs=-1, verbose=1)(
    delayed(extract_additional_features)(i, mode='test') for i in test_pet_ids)

test_dfs_sentiment = [x[0] for x in dfs_test if isinstance(x[0], pd.DataFrame)]
test_dfs_metadata = [x[1] for x in dfs_test if isinstance(x[1], pd.DataFrame)]

test_dfs_sentiment = pd.concat(test_dfs_sentiment, ignore_index=True, sort=False)
test_dfs_metadata = pd.concat(test_dfs_metadata, ignore_index=True, sort=False)

print(test_dfs_sentiment.shape, test_dfs_metadata.shape)

In [ ]:
train_dfs_sentiment.sentiment_language.value_counts()

In [ ]:
train_dfs_sentiment.groupby(['PetID'])['sentiment_language'].apply(lambda x: x.mode()).reset_index().drop(columns='level_1')

### group extracted features by PetID:

In [ ]:
aggregates = ['sum', 'mean', 'var', 'max', 'min']
sent_agg = ['sum', 'mean', 'var', 'max', 'min']


# Train
# まず desc は別にする
train_metadata_desc = train_dfs_metadata.groupby(['PetID'])['metadata_annots_top_desc'].unique()
train_metadata_desc = train_metadata_desc.reset_index()
train_metadata_desc[
    'metadata_annots_top_desc'] = train_metadata_desc[
    'metadata_annots_top_desc'].apply(lambda x: ' '.join(x))

#  desc 落として、aggregate
prefix = 'metadata'
train_metadata_gr = train_dfs_metadata.drop(['metadata_annots_top_desc'], axis=1)
for i in train_metadata_gr.columns:
    if 'PetID' not in i:
        train_metadata_gr[i] = train_metadata_gr[i].astype(float)
train_metadata_gr = train_metadata_gr.groupby(['PetID']).agg(aggregates)
train_metadata_gr.columns = pd.Index([f'{c[0]}_{c[1].upper()}' for c in train_metadata_gr.columns.tolist()])
train_metadata_gr = train_metadata_gr.reset_index()


train_sentiment_desc = train_dfs_sentiment.groupby(['PetID'])['sentiment_entities'].unique()
train_sentiment_desc = train_sentiment_desc.reset_index()
train_sentiment_desc[
    'sentiment_entities'] = train_sentiment_desc[
    'sentiment_entities'].apply(lambda x: ' '.join(x))

train_dfs_lang = train_dfs_sentiment.groupby(['PetID'])['sentiment_language'].apply(lambda x: x.mode()).reset_index().drop(columns='level_1')

prefix = 'sentiment'
train_sentiment_gr = train_dfs_sentiment.drop(['sentiment_entities','sentiment_language'], axis=1)
for i in train_sentiment_gr.columns:
    if 'PetID' not in i:
        train_sentiment_gr[i] = train_sentiment_gr[i].astype(float)
train_sentiment_gr = train_sentiment_gr.groupby(['PetID']).agg(sent_agg)
train_sentiment_gr.columns = pd.Index([f'{c[0]}_{c[1].upper()}' for c in train_sentiment_gr.columns.tolist()])
train_sentiment_gr = train_sentiment_gr.reset_index()


# Test
test_metadata_desc = test_dfs_metadata.groupby(['PetID'])['metadata_annots_top_desc'].unique()
test_metadata_desc = test_metadata_desc.reset_index()
test_metadata_desc[
    'metadata_annots_top_desc'] = test_metadata_desc[
    'metadata_annots_top_desc'].apply(lambda x: ' '.join(x))


prefix = 'metadata'
test_metadata_gr = test_dfs_metadata.drop(['metadata_annots_top_desc'], axis=1)
for i in test_metadata_gr.columns:
    if 'PetID' not in i:
        test_metadata_gr[i] = test_metadata_gr[i].astype(float)
test_metadata_gr = test_metadata_gr.groupby(['PetID']).agg(aggregates)
test_metadata_gr.columns = pd.Index([f'{c[0]}_{c[1].upper()}' for c in test_metadata_gr.columns.tolist()])
test_metadata_gr = test_metadata_gr.reset_index()


test_sentiment_desc = test_dfs_sentiment.groupby(['PetID'])['sentiment_entities'].unique()
test_sentiment_desc = test_sentiment_desc.reset_index()
test_sentiment_desc[
    'sentiment_entities'] = test_sentiment_desc[
    'sentiment_entities'].apply(lambda x: ' '.join(x))

test_dfs_lang = test_dfs_sentiment.groupby(['PetID'])['sentiment_language'].apply(lambda x: x.mode()).reset_index().drop(columns='level_1')

prefix = 'sentiment'
test_sentiment_gr = test_dfs_sentiment.drop(['sentiment_entities', 'sentiment_language'], axis=1)
for i in test_sentiment_gr.columns:
    if 'PetID' not in i:
        test_sentiment_gr[i] = test_sentiment_gr[i].astype(float)
test_sentiment_gr = test_sentiment_gr.groupby(['PetID']).agg(sent_agg)
test_sentiment_gr.columns = pd.Index([f'{c[0]}_{c[1].upper()}' for c in test_sentiment_gr.columns.tolist()])
test_sentiment_gr = test_sentiment_gr.reset_index()

In [ ]:
test_metadata_gr.describe()

### merge processed DFs with base train/test DF:

In [ ]:
# Train merges:
train_proc = train.copy()
train_proc = train_proc.merge(
    train_sentiment_gr, how='left', on='PetID')
train_proc = train_proc.merge(
    train_metadata_gr, how='left', on='PetID')
train_proc = train_proc.merge(
    train_metadata_desc, how='left', on='PetID')
train_proc = train_proc.merge(
    train_sentiment_desc, how='left', on='PetID')
train_proc = train_proc.merge(
    train_dfs_lang, how='left', on='PetID')

# Test merges:
test_proc = test.copy()
test_proc = test_proc.merge(
    test_sentiment_gr, how='left', on='PetID')
test_proc = test_proc.merge(
    test_metadata_gr, how='left', on='PetID')
test_proc = test_proc.merge(
    test_metadata_desc, how='left', on='PetID')
test_proc = test_proc.merge(
    test_sentiment_desc, how='left', on='PetID')
test_proc = test_proc.merge(
    test_dfs_lang, how='left', on='PetID')

print(train_proc.shape, test_proc.shape)
assert train_proc.shape[0] == train.shape[0]
assert test_proc.shape[0] == test.shape[0]

In [ ]:
train_breed_main = train_proc[['Breed1']].merge(
    labels_breed, how='left',
    left_on='Breed1', right_on='BreedID',
    suffixes=('', '_main_breed'))

train_breed_main = train_breed_main.iloc[:, 2:]
train_breed_main = train_breed_main.add_prefix('main_breed_')

train_breed_second = train_proc[['Breed2']].merge(
    labels_breed, how='left',
    left_on='Breed2', right_on='BreedID',
    suffixes=('', '_second_breed'))

train_breed_second = train_breed_second.iloc[:, 2:]
train_breed_second = train_breed_second.add_prefix('second_breed_')


train_proc = pd.concat(
    [train_proc, train_breed_main, train_breed_second], axis=1)


test_breed_main = test_proc[['Breed1']].merge(
    labels_breed, how='left',
    left_on='Breed1', right_on='BreedID',
    suffixes=('', '_main_breed'))

test_breed_main = test_breed_main.iloc[:, 2:]
test_breed_main = test_breed_main.add_prefix('main_breed_')

test_breed_second = test_proc[['Breed2']].merge(
    labels_breed, how='left',
    left_on='Breed2', right_on='BreedID',
    suffixes=('', '_second_breed'))

test_breed_second = test_breed_second.iloc[:, 2:]
test_breed_second = test_breed_second.add_prefix('second_breed_')


test_proc = pd.concat(
    [test_proc, test_breed_main, test_breed_second], axis=1)

print(train_proc.shape, test_proc.shape)

In [ ]:
X = pd.concat([train_proc, test_proc], ignore_index=True, sort=False)

In [ ]:
X.dtypes

In [ ]:
X_temp = X.copy()

text_columns = ['Description', 'metadata_annots_top_desc', 'sentiment_entities']
categorical_columns = ['main_breed_BreedName', 'second_breed_BreedName', 'sentiment_language']

to_drop_columns = ['PetID', 'Name', 'RescuerID']

In [ ]:
rescuer_count = X.groupby(['RescuerID'])['PetID'].count().reset_index()
rescuer_count.columns = ['RescuerID', 'RescuerID_COUNT']

X_temp = X_temp.merge(rescuer_count, how='left', on='RescuerID')

In [ ]:
for i in categorical_columns:
    X_temp.loc[:, i] = pd.factorize(X_temp.loc[:, i])[0]

In [ ]:
X['sentiment_language'].value_counts()

In [ ]:
X_temp['sentiment_language'].value_counts()

In [ ]:
X_text = X_temp[text_columns]

for i in X_text.columns:
    X_text.loc[:, i] = X_text.loc[:, i].fillna('none')

In [ ]:
X_temp['Length_Description'] = X_text['Description'].map(len)
X_temp['Length_metadata_annots_top_desc'] = X_text['metadata_annots_top_desc'].map(len)
X_temp['Lengths_sentiment_entities'] = X_text['sentiment_entities'].map(len)

### TFIDF

In [ ]:
n_components = 16
text_features = []

# Generate text features:
for i in X_text.columns:
    
    # Initialize decomposition methods:
    print(f'generating features from: {i}')
    tfv = TfidfVectorizer(min_df=2,  max_features=None,
                          strip_accents='unicode', analyzer='word', token_pattern=r'(?u)\b\w+\b',
                          ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1)
    svd_ = TruncatedSVD(
        n_components=n_components, random_state=1337)
    
    tfidf_col = tfv.fit_transform(X_text.loc[:, i].values)
    
    svd_col = svd_.fit_transform(tfidf_col)
    svd_col = pd.DataFrame(svd_col)
    svd_col = svd_col.add_prefix('TFIDF_{}_'.format(i))
    
    text_features.append(svd_col)
    
text_features = pd.concat(text_features, axis=1)

X_temp = pd.concat([X_temp, text_features], axis=1)

for i in X_text.columns:
    X_temp = X_temp.drop(i, axis=1)

### Merge image features

In [ ]:
X_temp = X_temp.merge(img_features, how='left', on='PetID')

### Add image_size features

In [ ]:
from PIL import Image
train_df_ids = train[['PetID']]
test_df_ids = test[['PetID']]

debut = False
if debug:
    train_pet_ids = train_pet_ids[:1000]
    test_pet_ids = test_pet_ids[:500]

train_df_imgs = pd.DataFrame(train_image_files)
train_df_imgs.columns = ['image_filename']
train_imgs_pets = train_df_imgs['image_filename'].apply(lambda x: x.split(split_char)[-1].split('-')[0])

test_df_imgs = pd.DataFrame(test_image_files)
test_df_imgs.columns = ['image_filename']
test_imgs_pets = test_df_imgs['image_filename'].apply(lambda x: x.split(split_char)[-1].split('-')[0])

train_df_imgs = train_df_imgs.assign(PetID=train_imgs_pets)
test_df_imgs = test_df_imgs.assign(PetID=test_imgs_pets)

def getSize(filename):
    st = os.stat(filename)
    return st.st_size

def getDimensions(filename):
    img_size = Image.open(filename).size
    return img_size 

train_df_imgs['image_size'] = train_df_imgs['image_filename'].apply(getSize)
train_df_imgs['temp_size'] = train_df_imgs['image_filename'].apply(getDimensions)
train_df_imgs['width'] = train_df_imgs['temp_size'].apply(lambda x : x[0])
train_df_imgs['height'] = train_df_imgs['temp_size'].apply(lambda x : x[1])
train_df_imgs = train_df_imgs.drop(['temp_size'], axis=1)

test_df_imgs['image_size'] = test_df_imgs['image_filename'].apply(getSize)
test_df_imgs['temp_size'] = test_df_imgs['image_filename'].apply(getDimensions)
test_df_imgs['width'] = test_df_imgs['temp_size'].apply(lambda x : x[0])
test_df_imgs['height'] = test_df_imgs['temp_size'].apply(lambda x : x[1])
test_df_imgs = test_df_imgs.drop(['temp_size'], axis=1)

aggs = {
    'image_size': ['sum', 'mean', 'var', 'max'],
    'width': ['sum', 'mean', 'var', 'max'],
    'height': ['sum', 'mean', 'var', 'max'],
}

agg_train_imgs = train_df_imgs.groupby('PetID').agg(aggs)
new_columns = [
    k + '_' + agg for k in aggs.keys() for agg in aggs[k]
]
agg_train_imgs.columns = new_columns
agg_train_imgs = agg_train_imgs.reset_index()

agg_test_imgs = test_df_imgs.groupby('PetID').agg(aggs)
new_columns = [
    k + '_' + agg for k in aggs.keys() for agg in aggs[k]
]
agg_test_imgs.columns = new_columns
agg_test_imgs = agg_test_imgs.reset_index()

agg_imgs = pd.concat([agg_train_imgs, agg_test_imgs], axis=0).reset_index(drop=True)

In [ ]:
X_temp = X_temp.merge(agg_imgs, how='left', on='PetID')

### Drop ID, name and rescuerID

In [ ]:
X_temp = X_temp.drop(to_drop_columns, axis=1)

In [ ]:
X_train = X_temp.loc[np.isfinite(X_temp.AdoptionSpeed), :]
X_test = X_temp.loc[~np.isfinite(X_temp.AdoptionSpeed), :]

X_test = X_test.drop(['AdoptionSpeed'], axis=1)

assert X_train.shape[0] == train.shape[0]
assert X_test.shape[0] == test.shape[0]

train_cols = X_train.columns.tolist()
train_cols.remove('AdoptionSpeed')

test_cols = X_test.columns.tolist()

assert np.all(train_cols == test_cols)

In [ ]:
X_train_non_null = X_train.fillna(-1)
X_test_non_null = X_test.fillna(-1)

In [ ]:
X_train_non_null.isnull().any().any(), X_test_non_null.isnull().any().any()

In [ ]:
X_train_non_null.shape, X_test_non_null.shape

In [ ]:
def target_encoding(count_offset, X_te_train, X_te_valid, X_te_test, sigm_scale_divider=5):
    categorical_column_list = ["Breed1", "Breed2", "Gender", "Color1", "Color2", "Color3", "Health", "State",
                               'sentiment_language']

    target_encoding_columns = []
    mean_all = X_te_train.AdoptionSpeed.mean()
    for categ_column in categorical_column_list:
        df_categorical = X_te_train[[categ_column, "AdoptionSpeed"]]
        df_categorical = df_categorical.fillna(-1).astype(int)

        te_df = df_categorical.groupby(categ_column).mean()
        df_count = df_categorical.groupby(categ_column).count()
        sigmoid_scale = (count_offset - 1) / sigm_scale_divider

        smoothing_coef = 1 / (1 + np.exp(-(df_count - count_offset) / sigmoid_scale))


        target_enc_df = mean_all * (1 - smoothing_coef) + te_df * smoothing_coef
        target_enc_df = target_enc_df.add_prefix(f'TargetEncoding_{categ_column}_')

        X_te_train = pd.merge(X_te_train, target_enc_df.reset_index(), on=categ_column, how='left')
        X_te_valid = pd.merge(X_te_valid, target_enc_df.reset_index(), on=categ_column, how='left')
        X_te_test = pd.merge(X_te_test, target_enc_df.reset_index(), on=categ_column, how='left')
    return X_te_train, X_te_valid, X_te_test

In [ ]:
import scipy as sp

from collections import Counter
from functools import partial
from math import sqrt

from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import confusion_matrix as sk_cmatrix


# FROM: https://www.kaggle.com/myltykritik/simple-lgbm-image-features

# The following 3 functions have been taken from Ben Hamner's github repository
# https://github.com/benhamner/Metrics
def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)

### OptimizeRounder from [OptimizedRounder() - Improved](https://www.kaggle.com/naveenasaithambi/optimizedrounder-improved)

In [ ]:
def rounder(y, thresholds=[0.5, 1.5, 2.5, 3.5]):
    return np.digitize(y, thresholds)

class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = [0.5, 1.5, 2.5, 3.5]

    def _loss(self, coef, X, y):
        X_p = rounder(X, np.sort(coef))
        ll = quadratic_weighted_kappa(y, X_p)
        return -ll

    def fit(self, X, y):
        coef = self.coef_
        for it1 in range(10):
            for idx in range(4):
                golden1 = 2/(3+np.sqrt(5))
                golden2 = 1 - golden1
                a = 0.0 if idx-1 < 0 else coef[idx-1]
                b = 4.0 if idx+1 > 3 else coef[idx+1]
                coef[idx] = a + (b-a)*golden1
                la = self._loss(coef, X, y)
                coef[idx] = a + (b-a)*golden2
                lb = self._loss(coef, X, y)
                for it2 in range(20):
                    if la > lb:
                        a = a + (b-a)*golden1
                        coef[idx] = a + (b-a)*golden2
                        la = lb
                        lb = self._loss(coef, X, y)
                    else:
                        b = a + (b-a)*golden2
                        coef[idx] = a + (b-a)*golden1
                        lb = la
                        la = self._loss(coef, X, y)
        self.coef_ = coef

    def predict(self, X, coef):
        return rounder(X, coef)

    def coefficients(self):
        return self.coef_

In [ ]:
import random
from collections import Counter, defaultdict
def stratified_group_k_fold(X, y, groups, k, seed=None):
    labels_num = np.max(y) + 1
    y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
    y_distr = Counter()
    for label, g in zip(y, groups):
        y_counts_per_group[g][label] += 1
        y_distr[label] += 1

    y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
    groups_per_fold = defaultdict(set)

    def eval_y_counts_per_fold(y_counts, fold):
        y_counts_per_fold[fold] += y_counts
        std_per_label = []
        for label in range(labels_num):
            label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(k)])
            std_per_label.append(label_std)
        y_counts_per_fold[fold] -= y_counts
        return np.mean(std_per_label)
    
    groups_and_y_counts = list(y_counts_per_group.items())
    random.Random(seed).shuffle(groups_and_y_counts)

    for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
        best_fold = None
        min_eval = None
        for i in range(k):
            fold_eval = eval_y_counts_per_fold(y_counts, i)
            if min_eval is None or fold_eval < min_eval:
                min_eval = fold_eval
                best_fold = i
        y_counts_per_fold[best_fold] += y_counts
        groups_per_fold[best_fold].add(g)

    all_groups = set(groups)
    for i in range(k):
        train_groups = all_groups - groups_per_fold[i]
        test_groups = groups_per_fold[i]

        train_indices = [i for i, g in enumerate(groups) if g in train_groups]
        test_indices = [i for i, g in enumerate(groups) if g in test_groups]

        yield train_indices, test_indices

## Train model

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GroupKFold, StratifiedKFold
train_gid = train.RescuerID.values

def run_xgb(params, X_train, X_test, cv):
    n_splits = 10
    verbose_eval = 10000
    num_rounds = 5000
    early_stop = 500
    
    try:
        kf = cv(n_splits=n_splits, shuffle=True)
    except:
        kf = cv(n_splits=n_splits)

    oof_train = np.zeros((X_train.shape[0]))
    oof_test = np.zeros((X_test.shape[0], n_splits))

    i = 0
    importance_list = []
    
    X_test_original = X_test.copy()
    for train_idx, valid_idx in tqdm(kf.split(X_train, X_train['AdoptionSpeed'].values, train_gid)):
        X_tr = X_train.iloc[train_idx, :]
        X_val = X_train.iloc[valid_idx, :]
        
        X_tr, X_val, X_test = target_encoding(params['count_offset'], X_tr, X_val, X_test_original, params['sigm_scale_divider'])
        y_tr = X_tr['AdoptionSpeed'].values
        X_tr = X_tr.drop(['AdoptionSpeed'], axis=1)

        y_val = X_val['AdoptionSpeed'].values
        X_val = X_val.drop(['AdoptionSpeed'], axis=1)

        d_train = xgb.DMatrix(data=X_tr, label=y_tr, feature_names=X_tr.columns)
        d_valid = xgb.DMatrix(data=X_val, label=y_val, feature_names=X_val.columns)

        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        model = xgb.train(dtrain=d_train, num_boost_round=num_rounds, evals=watchlist,
                         early_stopping_rounds=early_stop, verbose_eval=verbose_eval, params=params)

        valid_pred = model.predict(xgb.DMatrix(X_val, feature_names=X_val.columns), ntree_limit=model.best_ntree_limit)
        test_pred = model.predict(xgb.DMatrix(X_test, feature_names=X_test.columns), ntree_limit=model.best_ntree_limit)

        oof_train[valid_idx] = valid_pred
        oof_test[:, i] = test_pred
        
        importance_list.append(model.get_score(importance_type='gain'))

        i += 1
    return model, oof_train, oof_test.mean(axis=1)

def run_xgb_stratified_group(params, X_train, X_test):
    n_splits = 10
    verbose_eval = 10000
    num_rounds = 5000
    early_stop = 500


    oof_train = np.zeros((X_train.shape[0]))
    oof_test = np.zeros((X_test.shape[0], n_splits))

    i = 0
    importance_list = []
    
    X_test_original = X_test.copy()
    for train_idx, valid_idx in tqdm(stratified_group_k_fold(X_train, X_train['AdoptionSpeed'].values.astype(np.int8), train_gid, k=n_splits)):
        X_tr = X_train.iloc[train_idx, :]
        X_val = X_train.iloc[valid_idx, :]
        
        X_tr, X_val, X_test = target_encoding(params['count_offset'], X_tr, X_val, X_test_original, params['sigm_scale_divider'])
        y_tr = X_tr['AdoptionSpeed'].values
        X_tr = X_tr.drop(['AdoptionSpeed'], axis=1)

        y_val = X_val['AdoptionSpeed'].values
        X_val = X_val.drop(['AdoptionSpeed'], axis=1)

        d_train = xgb.DMatrix(data=X_tr, label=y_tr, feature_names=X_tr.columns)
        d_valid = xgb.DMatrix(data=X_val, label=y_val, feature_names=X_val.columns)

        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        model = xgb.train(dtrain=d_train, num_boost_round=num_rounds, evals=watchlist,
                         early_stopping_rounds=early_stop, verbose_eval=verbose_eval, params=params)

        valid_pred = model.predict(xgb.DMatrix(X_val, feature_names=X_val.columns), ntree_limit=model.best_ntree_limit)
        test_pred = model.predict(xgb.DMatrix(X_test, feature_names=X_test.columns), ntree_limit=model.best_ntree_limit)

        oof_train[valid_idx] = valid_pred
        oof_test[:, i] = test_pred
        
        importance_list.append(model.get_score(importance_type='gain'))

        i += 1
    return model, oof_train, oof_test.mean(axis=1)

In [ ]:
oof_train_list = []
oof_test_list = []
for i_seed in range(Config.num_seed_stratified):
    print(i_seed)
    gc.collect()
    xgb_params = {
        'eval_metric': 'rmse',
        'seed': i_seed,
        'eta': 0.02042285921574787,
         'gamma': 0.0006332233837072564,
         'max_depth': 4,
         'min_child_weight': 12,
         'subsample': 0.8928595200024736,
         'colsample_bytree': 0.3894222269910513,
        'colsample_bylevel': 0.7,
        'tree_method': 'gpu_hist',
        'device': 'gpu',
        'silent': 1,
        'count_offset': 40,
        'sigm_scale_divider': 50
    }
    model, oof_train, oof_test = run_xgb(xgb_params, X_train_non_null, X_test_non_null, StratifiedKFold)
    oof_train_list.append(oof_train)
    oof_test_list.append(oof_test)
    
for i_seed in range(Config.num_seed_group):
    print(i_seed)
    gc.collect()
    xgb_params = {
        'eval_metric': 'rmse',
        'seed': i_seed,
        'eta': 0.02042285921574787,
         'gamma': 0.0006332233837072564,
         'max_depth': 4,
         'min_child_weight': 12,
         'subsample': 0.8928595200024736,
         'colsample_bytree': 0.3894222269910513,
        'colsample_bylevel': 0.7,
        'tree_method': 'gpu_hist',
        'device': 'gpu',
        'silent': 1,
        'count_offset': 40,
        'sigm_scale_divider': 50
    }
    model, oof_train, oof_test = run_xgb(xgb_params, X_train_non_null, X_test_non_null, GroupKFold)
    oof_train_list.append(oof_train)
    oof_test_list.append(oof_test)
    
for i_seed in range(Config.num_seed_stratified_group):
    print(i_seed)
    gc.collect()
    xgb_params = {
        'eval_metric': 'rmse',
        'seed': i_seed,
        'eta': 0.02042285921574787,
         'gamma': 0.0006332233837072564,
         'max_depth': 4,
         'min_child_weight': 12,
         'subsample': 0.8928595200024736,
         'colsample_bytree': 0.3894222269910513,
        'colsample_bylevel': 0.7,
        'tree_method': 'gpu_hist',
        'device': 'gpu',
        'silent': 1,
        'count_offset': 40,
        'sigm_scale_divider': 50
    }
    model, oof_train, oof_test = run_xgb_stratified_group(xgb_params, X_train_non_null, X_test_non_null)
    oof_train_list.append(oof_train)
    oof_test_list.append(oof_test)
oof_train = np.mean(oof_train_list, axis=0)
oof_test = np.mean(oof_test_list, axis=0)


In [ ]:
xgb_oof_train, xgb_oof_test = oof_train, oof_test

In [ ]:
def plot_pred(pred):
    sns.distplot(pred, kde=True, hist_kws={'range': [0, 5]})

In [ ]:
plot_pred(oof_train)

In [ ]:
plot_pred(oof_test)

In [ ]:
optR = OptimizedRounder()
optR.fit(oof_train, X_train['AdoptionSpeed'].values)
coefficients = optR.coefficients()
valid_pred = optR.predict(oof_train, coefficients)
qwk = quadratic_weighted_kappa(X_train['AdoptionSpeed'].values, valid_pred)
print("QWK = ", qwk)

coefficients_ = optR.coefficients()
train_predictions = optR.predict(oof_train, coefficients_).astype(np.int8)
print(f'train pred distribution: {Counter(train_predictions)}')
test_predictions = optR.predict(oof_test, coefficients_).astype(np.int8)
print(f'test pred distribution: {Counter(test_predictions)}')

# LightGBM

In [ ]:
import lightgbm as lgb

In [ ]:
def run_lgbm(params, X_train, X_test, cv, plot=False):
    n_splits = 10
    early_stop = 500
    verbose_eval = 10000

    try:
        kf = cv(n_splits=n_splits, shuffle=True)
    except:
        kf = cv(n_splits=n_splits)

    oof_train = np.zeros((X_train.shape[0]))
    oof_test = np.zeros((X_test.shape[0], n_splits))

    i = 0
    importance_list = []
    
    X_test_original = X_test.copy()
    for train_idx, valid_idx in tqdm(kf.split(X_train, X_train['AdoptionSpeed'].values, train_gid)):
        X_tr = X_train.iloc[train_idx, :]
        X_val = X_train.iloc[valid_idx, :]
        
        X_tr, X_val, X_test = target_encoding(params['count_offset'], X_tr, X_val, X_test_original, 
                                              params['sigm_scale_divider'])
        y_tr = X_tr['AdoptionSpeed'].values
        X_tr = X_tr.drop(['AdoptionSpeed'], axis=1)

        y_val = X_val['AdoptionSpeed'].values
        X_val = X_val.drop(['AdoptionSpeed'], axis=1)

        d_train = lgb.Dataset(X_tr, label=y_tr, feature_name=X_tr.columns.tolist())
        d_valid = lgb.Dataset(X_val, label=y_val, feature_name=X_val.columns.tolist())

        model = lgb.train(params, d_train, valid_sets=[d_valid], early_stopping_rounds=early_stop,
                         verbose_eval=verbose_eval)

        valid_pred = model.predict(X_val, num_iteration=model.best_iteration)
        test_pred = model.predict(X_test, num_iteration=model.best_iteration)

        oof_train[valid_idx] = valid_pred
        oof_test[:, i] = test_pred
        if plot:
            train_pred = model.predict(X_tr, num_iteration=model.best_iteration)
            plot_pred(train_pred)
            plot_pred(valid_pred)
            plot_pred(test_pred)
        

        i += 1
    return model, oof_train, oof_test.mean(axis=1)


def run_lgbm_stratified_group(params, X_train, X_test, plot=False):
    n_splits = 10
    early_stop = 500
    verbose_eval = 10000

    oof_train = np.zeros((X_train.shape[0]))
    oof_test = np.zeros((X_test.shape[0], n_splits))

    i = 0
    importance_list = []
    
    X_test_original = X_test.copy()
    for train_idx, valid_idx in tqdm(stratified_group_k_fold(X_train, X_train['AdoptionSpeed'].values.astype(np.int8), train_gid, k=n_splits)):
        X_tr = X_train.iloc[train_idx, :]
        X_val = X_train.iloc[valid_idx, :]
        
        X_tr, X_val, X_test = target_encoding(params['count_offset'], X_tr, X_val, X_test_original, 
                                              params['sigm_scale_divider'])
        y_tr = X_tr['AdoptionSpeed'].values
        X_tr = X_tr.drop(['AdoptionSpeed'], axis=1)

        y_val = X_val['AdoptionSpeed'].values
        X_val = X_val.drop(['AdoptionSpeed'], axis=1)

        d_train = lgb.Dataset(X_tr, label=y_tr, feature_name=X_tr.columns.tolist())
        d_valid = lgb.Dataset(X_val, label=y_val, feature_name=X_val.columns.tolist())

        model = lgb.train(params, d_train, valid_sets=[d_valid], early_stopping_rounds=early_stop,
                         verbose_eval=verbose_eval)

        valid_pred = model.predict(X_val, num_iteration=model.best_iteration)
        test_pred = model.predict(X_test, num_iteration=model.best_iteration)

        oof_train[valid_idx] = valid_pred
        oof_test[:, i] = test_pred
        if plot:
            train_pred = model.predict(X_tr, num_iteration=model.best_iteration)
            plot_pred(train_pred)
            plot_pred(valid_pred)
            plot_pred(test_pred)
        

        i += 1
    return model, oof_train, oof_test.mean(axis=1)

In [ ]:
oof_train_list = []
oof_test_list = []

for i_seed in tqdm(range(3)):
    gc.collect()

    lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'rmse',
    'eta': 0.01235408383482946,
    'subsample': 0.90892648949676, 
    'colsample_bytree': 0.3595323228044939,
    'min_split_gain': 0.0017213784304419208,
    'min_child_weight': 2.489294481302874e-05, 
    'min_child_samples': 3,
        'n_estimators': 1838,
        'max_depth': 12,
        'count_offset': 21.197455354403893,
        'sigm_scale_divider': 49.03532266115622,
        'bagging_freq': 4,
    'verbose': -1,
    'silent': 1,
    'n_jobs': -1,
    'seed': i_seed
    }


    model, oof_train, oof_test = run_lgbm(lgb_params, X_train_non_null, X_test_non_null, StratifiedKFold)
    oof_train_list.append(oof_train)
    oof_test_list.append(oof_test)
    
for i_seed in tqdm(range(3)):
    gc.collect()

    lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'rmse',
    'eta': 0.01235408383482946,
    'subsample': 0.90892648949676, 
    'colsample_bytree': 0.3595323228044939,
    'min_split_gain': 0.0017213784304419208,
    'min_child_weight': 2.489294481302874e-05, 
    'min_child_samples': 3,
        'n_estimators': 1838,
        'max_depth': 12,
        'count_offset': 21.197455354403893,
        'sigm_scale_divider': 49.03532266115622,
        'bagging_freq': 4,
    'verbose': -1,
    'silent': 1,
    'n_jobs': -1,
    'seed': i_seed
    }


    model, oof_train, oof_test = run_lgbm(lgb_params, X_train_non_null, X_test_non_null, GroupKFold)
    oof_train_list.append(oof_train)
    oof_test_list.append(oof_test)
    
for i_seed in tqdm(range(3)):
    gc.collect()

    lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'rmse',
    'eta': 0.01235408383482946,
    'subsample': 0.90892648949676, 
    'colsample_bytree': 0.3595323228044939,
    'min_split_gain': 0.0017213784304419208,
    'min_child_weight': 2.489294481302874e-05, 
    'min_child_samples': 3,
        'n_estimators': 1838,
        'max_depth': 12,
        'count_offset': 21.197455354403893,
        'sigm_scale_divider': 49.03532266115622,
        'bagging_freq': 4,
    'verbose': -1,
    'silent': 1,
    'n_jobs': -1,
    'seed': i_seed
    }


    model, oof_train, oof_test = run_lgbm_stratified_group(lgb_params, X_train_non_null, X_test_non_null)
    oof_train_list.append(oof_train)
    oof_test_list.append(oof_test)
oof_train = np.mean(oof_train_list, axis=0)
oof_test = np.mean(oof_test_list, axis=0)

In [ ]:
plot_pred(oof_train)

In [ ]:
plot_pred(oof_test)

In [ ]:
optR = OptimizedRounder()
optR.fit(oof_train, X_train['AdoptionSpeed'].values)
coefficients = optR.coefficients()
valid_pred = optR.predict(oof_train, coefficients)
qwk = quadratic_weighted_kappa(X_train['AdoptionSpeed'].values, valid_pred)
print("QWK = ", qwk)

coefficients_ = optR.coefficients()
train_predictions = optR.predict(oof_train, coefficients_).astype(np.int8)
print(f'train pred distribution: {Counter(train_predictions)}')
test_predictions = optR.predict(oof_test, coefficients_).astype(np.int8)
print(f'test pred distribution: {Counter(test_predictions)}')

# LightGBM + Xgboost

In [ ]:
lgb_oof_train = oof_train
lgb_oof_test = oof_test

In [ ]:
best_qwk = 0
for weight in np.arange(0, 1.01, 0.1):
    print(weight)
    oof_train = weight * xgb_oof_train + (1 - weight) * lgb_oof_train
    oof_test = weight * xgb_oof_test + (1 - weight) * lgb_oof_test

    plot_pred(oof_train)

    plot_pred(oof_test)

    optR = OptimizedRounder()
    optR.fit(oof_train, X_train['AdoptionSpeed'].values)
    coefficients = optR.coefficients()
    valid_pred = optR.predict(oof_train, coefficients)
    qwk = quadratic_weighted_kappa(X_train['AdoptionSpeed'].values, valid_pred)
    print("QWK = ", qwk)

    coefficients_ = optR.coefficients()
    train_predictions = optR.predict(oof_train, coefficients_).astype(np.int8)
    print(f'train pred distribution: {Counter(train_predictions)}')
    test_predictions = optR.predict(oof_test, coefficients_).astype(np.int8)
    print(f'test pred distribution: {Counter(test_predictions)}')
    
    if qwk > best_qwk:
        best_weight = weight
        best_qwk = qwk
        best_test_predictions = test_predictions

print(best_weight, best_qwk)

In [ ]:
submission = pd.DataFrame({'PetID': test['PetID'].values, 'AdoptionSpeed': best_test_predictions})
submission.to_csv('submission.csv', index=False)
submission.head()